In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import keras
import os 
from PIL import Image
import cv2 as cv 

In [2]:
from keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

### Data Import

In [3]:
#path to every images for train and test

cat_train = 'training_set/training_set/cats/'

cat_test = 'test_set/test_set/cats/'

dog_train = 'training_set/training_set/dogs/'

dog_test = 'test_set/test_set/dogs/'


train_data = 'training_set/training_set/'
test_data = 'test_set/test_set/'

In [4]:
#view total images inside every directory

print("Total cats for training = ", len(os.listdir(cat_train)))
print("Total cats for testing = ", len(os.listdir(cat_test)))
print("Total dogs for training = ", len(os.listdir(dog_train)))
print("Total dogs for testing = ", len(os.listdir(dog_test)))


Total cats for training =  4000
Total cats for testing =  1011
Total dogs for training =  4005
Total dogs for testing =  1012


In [5]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
for layer in resnet.layers:
    layer.trainable = False,

In [6]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
)

In [8]:
target = (224,224)
batch = 32
epochs = 10

In [9]:
train_generator = train_datagen.flow_from_directory(
    directory=train_data,
    target_size=target,
    batch_size=batch,
    class_mode='binary',
    subset='training'
)


validation_generator = train_datagen.flow_from_directory(
    directory=train_data,
    target_size=target,
    batch_size=batch,
    class_mode='binary',
    subset='validation'
)



test_generator = test_datagen.flow_from_directory(
    directory=test_data,
    target_size=target,
    batch_size=batch,
    class_mode='binary'
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [10]:
x = resnet.output
x = GlobalAveragePooling2D()(x) 
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=resnet.input, outputs=predictions)

In [11]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10


  5/201 [..............................] - ETA: 1:30:55 - loss: 5.7752 - accuracy: 0.5562